In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from transformers import BertTokenizer, BertModel

/home/javier/Escritorio/HERIOT-WATT/F21DL - Data Mining and Machine Learning/group-coursework-jjmps/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_test = pd.read_csv("../../../data/health_fact/clean_dataset/test.csv")


In [3]:
df_test = df_test[~df_test['label'].isin(['unproven', 'mixture', 'snopes'])]

In [4]:
df_test.dropna(subset=["claim", "main_text", "label"], inplace=True)

In [5]:
missing_values = df_test.isna().sum()

print (f"Missing values {missing_values}")

Missing values Unnamed: 0.1      0
Unnamed: 0        0
claim_id          0
claim             0
date_published    0
explanation       0
fact_checkers     0
main_text         0
label             0
subjects          0
dtype: int64


In [6]:
print(len(df_test))

print (df_test["explanation"][400])

647
The proportion exposed to one or several ‘crimes against person’ was 11 percent in 2005. Subsequently, the proportion gradually decreased, and then stabilized at a level of around 9 percent. The last three years the trend is not as clear. In 2013 there was a rise to 11 percent, which was supposed to be temporary because the following year showed a decline to 9 percent. In 2015, however, the proportion is again 11 percent. The year-to-year variations in recent years is therefore large.


In [7]:

X = np.load("../../../data/health_fact/embeddings/embeddings_clean_train.npy")
Y = pd.read_csv("../../../data/health_fact/clean_dataset/train.csv")["label"].to_numpy()

In [8]:
print(len(X))
print (len(Y))
print (Y)

4991
4991
[False  True  True ...  True False False]


In [9]:

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [10]:
clf = tree.DecisionTreeClassifier()

In [11]:
clf = clf.fit(X, Y)

In [12]:
test_embeddings = np.array([])
good = 0
bad = 0
def transform_bool(value):
    if value == "false":
        return False
    return True

embedding_test = np.load("../../../data/health_fact/embeddings/embeddings_clean_test.npy")
# print (df_test["claim"][0])
data_length = len(df_test)
X_pred = df_test["main_text"]
Y_target = df_test["label"].apply(transform_bool)
for index, i in enumerate(embedding_test):
    #inputs = tokenizer(X_pred[i], return_tensors="pt", padding=True, truncation=True)
    #outputs = model(**inputs)
    #cls_embedding = outputs.last_hidden_state.mean(dim=1)
    # test_embeddings.append(cls_embedding.detach().numpy())
    output = clf.predict(np.array(i).reshape(1, -1))
    if output == Y_target[index]:
        good += 1
    else:
        bad += 1


In [13]:
print (good)
print (bad)
print (f"accuracy: {good / data_length}")

287
360
accuracy: 0.4435857805255023
